In [ ]:
import numpy as np
from tiatoolbox.wsicore.wsireader import VirtualWSIReader, WSIReader
import os
import matplotlib.pyplot as plt
import cv2
from utils import mm2_to_px, dist_to_px
import skimage
from scipy.spatial import Delaunay
from shapely.ops import polygonize, unary_union
import math
import shapely.geometry as geometry
import shapely
from PIL import Image, ImageDraw
import tifffile as tiff
from scipy.ndimage import zoom
from tiatoolbox.tools.patchextraction import (
    get_patch_extractor,
    SlidingWindowPatchExtractor,
)
from tiatoolbox.utils.misc import imwrite

# os.environ['PATH'] = "/opt/ASAP/bin" + ";" + os.environ['PATH']
import sys

sys.path.append("/opt/ASAP/bin")
from wholeslidedata.interoperability.asap.imagewriter import (
    WholeSlideMonochromeMaskWriter,
)
from config import DefaultConfig

In [22]:
torch_seg_out_path = DefaultConfig.seg_out_dir
wsi_dir = "/home/u1910100/Documents/Tiger_Data/wsitils/images"
asap_mask_dir = "/home/u1910100/Documents/Tiger_Data/wsitils/tissue-masks"


# wsi_name = "TC_S01_P000004_C0001_B103.tif"
wsi_name = "104S.tif"
wsi_name_without_ext = os.path.splitext(wsi_name)[0]

asap_mask_path = os.path.join(asap_mask_dir, f"{wsi_name_without_ext}_tissue.tif")

tumor_mask_path = os.path.join(torch_seg_out_path, f"{wsi_name_without_ext}_tumor.npy")
stroma_mask_path = os.path.join(
    torch_seg_out_path, f"{wsi_name_without_ext}_stroma.npy"
)

# reader = WSIReader.open(os.path.join(wsi_dir, wsi_name))
# thumb = reader.slide_thumbnail(resolution=0.3125, units="power")
# plt.imshow(thumb)
# plt.show()

In [ ]:
mask = WSIReader.open(asap_mask_path)
thumb = mask.slide_thumbnail(resolution=0.3125, units="power")[:, :, 0]
plt.imshow(thumb)
plt.show()
# factor = 64
# downsampled_image = zoom(mask, (1/factor, 1/factor))
# print(downsampled_image.shape)
# print(downsampled_image.dtype)

In [ ]:
tif = tiff.TiffFile(asap_mask_path)
print(len(tif.pages))
page = tif.pages[6]
mask = page.asarray()

plt.imshow(mask)
plt.show()

In [21]:
def convert_tissue_masks_for_l1(
    wsi_name_without_ext, tumor_mask_path, stroma_mask_path
):
    # Masks are at 5x resolution
    tumor_mask = np.load(tumor_mask_path)
    stroma_mask = np.load(stroma_mask_path)

    combined_mask = np.zeros_like(tumor_mask)
    combined_mask[np.where(tumor_mask == 1)] = 1
    combined_mask[np.where(stroma_mask == 1)] = 2

    mask_shape = combined_mask.shape
    patch_extractor = SlidingWindowPatchExtractor(
        input_img=combined_mask, patch_size=(512, 512)
    )

    tif_save_path = os.path.join(
        DefaultConfig.seg_out_dir, f"{wsi_name_without_ext}.tif"
    )
    writer = WholeSlideMonochromeMaskWriter()
    writer.write(
        path=tif_save_path,
        spacing=0.5,
        dimensions=(mask_shape[1] * 4, mask_shape[0] * 4),
        tile_shape=(512 * 4, 512 * 4),
    )
    for i, patch in enumerate(patch_extractor):
        # mask =
        mask = cv2.resize(
            patch[:, :, 0], (512 * 4, 512 * 4), interpolation=cv2.INTER_NEAREST
        ).astype("uint8")
        x_start, y_start = (
            patch_extractor.coordinate_list[i][0],
            patch_extractor.coordinate_list[i][1],
        )
        writer.write_tile(tile=mask, coordinates=(int(x_start) * 4, int(y_start) * 4))
    writer.save()

In [23]:
from matplotlib.colors import ListedColormap

cmap = ListedColormap(["black", "red", "green"])

convert_tissue_masks_for_l1(wsi_name_without_ext, tumor_mask_path, stroma_mask_path)

|2024-05-21|16:37:39.858| [WARNING] Raw data is None.
|2024-05-21|16:37:39.858| [WARNING] Unknown scale (no objective_power or mpp)


Creating: /home/u1910100/Documents/Tiger_Data/prediction/seg_out/104S.tif....
Spacing: 0.5
Dimensions: (55552, 36352)
Tile_shape: (2048, 2048)
Total time was 13539
Total reading time was 0
Total base writing time was 5274
Total pyramid downsampling time was 2
Total pyramid writing time was 8265
Total time determining min/max was 1014
